In [2]:
"""
Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в заголовке которых встречается хотя бы одно требуемое ключевое слово.
Эти слова определяем в начале кода в переменной, например:
KEYWORDS = ['python', 'парсинг']

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>
Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только заголовки статьи, но и весь текст статьи целиком.
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>
"""
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import re


def get_url():
    return 'https://habr.com/ru/all'


def get_soup(page=None):
    str_page = ''
    if page:
        str_page = f'page{page}/'
    url = f'{get_url()}/{str_page}'
    req = requests.get(url)
    time.sleep(0.2)
    return BeautifulSoup(req.text, 'html.parser')


habr_news = pd.DataFrame()
KEYWORDS = {'python', 'парсинг'}

next_page = 1
max_pg = 1
# производим поиск во всех доступных страницах
all_posts = []
while next_page <= max_pg:
    if next_page == 1:
        soup = get_soup()
    else:
        soup = get_soup(next_page)

    pg_list = soup.find_all('a', class_='tm-pagination__page')
    max_pg = int(re.sub(r"[^0-9]", '', pg_list[len(pg_list) - 1].text))
    next_page += 1

    posts = soup.find_all('a', class_='tm-article-snippet__title-link')
    for post in posts:
        post_list = set([x for x in post.find('span').text.split(' ')])
        if any(elem in KEYWORDS for elem in post_list):
            date = pd.to_datetime(post.parent.parent.find('span', class_='tm-article-snippet__datetime-published').
                                  find('time').
                                  get('datetime'),
                                  dayfirst=True).date()

            title = post.find('span').text
            link_ = f'https://habr.com{post.get("href")}'
            # print(f'date: {date}, title: {title}, link: {link_}')
            row = {'date': date, 'title': title, 'link': link_}
            habr_news = pd.concat([habr_news, pd.DataFrame([row])])


habr_news

,date,title,link
0,2021-12-10,"Загрузка, парсинг и визуализация данных без пр...",https://habr.com/ru/company/tablum/blog/594643/
0,2021-12-06,Дорожная карта по изучению python,https://habr.com/ru/post/593703/


In [6]:
"""
Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. 
Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи “скрытого” API. Внимательно изучите post-запросы.
"""

import requests
import time
import pandas as pd


url_ = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
header_ = {
    'Accept': 'application/json, text/plain, */*',
    'Content-Type': 'application/json;charset=UTF-8',
    'Host': 'identityprotection.avast.com',
    'Origin': 'https://www.avast.com',
    'Referer': 'https://www.avast.com/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}
params_ = {
    'emailAddresses': ['gena@mail.ru', 'gennadiy@gmail.com']
}
df = pd.DataFrame()
res = requests.post(url=url_, headers=header_, json=params_)
time.sleep(0.2)
print(res.status_code)
json_data = res.json()
if res.status_code == 200 and json_data is not None:
    for mail_ in json_data['summary']:
        for b in json_data['breaches']:
            if int(b) in json_data['summary'][mail_]['breaches']:            
                row = {'mail': mail_,
                       'publishDate': json_data['breaches'][b]['publishDate'],
                       'publishedSource': json_data['breaches'][b]['site'],
                       'description': json_data['breaches'][b]['description']
                       }
                df = pd.concat([df, pd.DataFrame([row])])

    print(df.head())
else:
    print(res.content)



200
           mail           publishDate  publishedSource  \
0  gena@mail.ru  2019-08-08T00:00:00Z      borealis.su   
0  gena@mail.ru  2020-07-23T00:00:00Z      wattpad.com   
0  gena@mail.ru  2019-12-05T00:00:00Z  wowprogress.com   
0  gena@mail.ru  2020-12-17T00:00:00Z    belarusgo.com   
0  gena@mail.ru  2020-12-17T00:00:00Z   forum.nedug.ru   

                                         description  
0  At an unconfirmed date, Russian Minecraft serv...  
0  In June 2020, the online writing community Wat...  
0  At an unconfirmed date, the World of Warcraft ...  
0  In November 2020, a collection of over 23,000 ...  
0  In November 2020, a collection of over 23,000 ...  
